In [1]:
!pip install git+https://github.com/hiredscorelabs/tamnun-ml

  Cloning https://github.com/hiredscorelabs/tamnun-ml to /tmp/pip-req-build-kuuvwuf3
  Running command git clone -q https://github.com/hiredscorelabs/tamnun-ml /tmp/pip-req-build-kuuvwuf3
     |████████████████████████████████| 13.9MB 3.4MB/s 
     |████████████████████████████████| 5.4MB 32.9MB/s 
     |████████████████████████████████| 143kB 48.5MB/s 
     |████████████████████████████████| 655kB 45.2MB/s 
     |████████████████████████████████| 1.0MB 32.4MB/s 
  Created wheel for tamnun: filename=tamnun-0.1.0-cp36-none-any.whl size=9440 sha256=b96140d6c9cb8ba19803f91570c9102afe6cc0a375d5bbc084aed6105220e2ea
  Stored in directory: /tmp/pip-ephem-wheel-cache-v5dq_2c0/wheels/3a/d7/c7/e7957cdb3f1e9185549672ac4389548eacf6b90e07c88a46c7
  Created wheel for regex: filename=regex-2019.6.8-cp36-cp36m-linux_x86_64.whl size=604153 sha256=691351b42ec4015e5e0e4899d712a564ea9be511a297b257a2cb799603ad4ef7
  Stored in directory: /root/.cache/pip/wheels/35/e4/80/abf3b33ba89cf65cd262af8a22a5a999cc28f

In [2]:
!git clone https://github.com/omilab/Neural-Sentiment-Analyzer-for-Modern-Hebrew

Cloning into 'Neural-Sentiment-Analyzer-for-Modern-Hebrew'...
remote: Enumerating objects: 33, done.
remote: Total 33 (delta 0), reused 0 (delta 0), pack-reused 33
Unpacking objects: 100% (33/33), done.


In [0]:
!mv Neural-Sentiment-Analyzer-for-Modern-Hebrew/data data

In [0]:
import codecs
import re
import numpy as np
from tamnun.bert import BertVectorizer, BertClassifier

In [0]:
def load_data(filename):
    data = list(codecs.open(filename, 'r', 'utf-8').readlines())
    x, y = zip(*[d.strip().split('\t') for d in data])
    x = np.asarray(list(x))
    #y = to_categorical(y, 3)
    
    return x, np.array(list(map(int, y)))

In [23]:
train_tokens, train_tags = load_data('data/token_train.tsv')
test_tokens, test_tags = load_data('data/token_test.tsv')
train_tokens.shape, train_tags.shape

((10244,), (10244,))

In [24]:
vectorizer = BertVectorizer(do_truncate=True, bert_model='bert-base-multilingual-cased').fit(train_tokens)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.
100%|██████████| 995526/995526 [00:00<00:00, 5661655.25B/s]


In [0]:
train_X = vectorizer.transform(train_tokens)

In [28]:
clf = BertClassifier(num_of_classes=3, bert_model_name='bert-base-multilingual-cased', lr=1e-5).fit(train_X, train_tags)

Epoch 1/5:
2560/2561 batch loss: 0.30547451972961426 avg loss: 0.41300118063190683
Epoch 2/5:
2560/2561 batch loss: 0.057198941707611084 avg loss: 0.24542082355083875
Epoch 3/5:
2560/2561 batch loss: 0.21260246634483337 avg loss: 0.1486284119258491
Epoch 4/5:
2560/2561 batch loss: 0.06112164258956909 avg loss: 0.09607804239331535
Epoch 5/5:
2560/2561 batch loss: 0.020296812057495117 avg loss: 0.0736653968104158


In [0]:
test_X = vectorizer.transform(test_tokens)

In [0]:
predicted = clf.predict(test_X)

In [0]:
from sklearn.metrics import accuracy_score, classification_report

In [34]:
print('Accuracy:', accuracy_score(test_tags, predicted))
print(classification_report(test_tags, predicted))

Accuracy: 0.928515625
              precision    recall  f1-score   support

           0       0.95      0.96      0.95      1698
           1       0.91      0.89      0.90       790
           2       0.62      0.61      0.62        72

   micro avg       0.93      0.93      0.93      2560
   macro avg       0.83      0.82      0.82      2560
weighted avg       0.93      0.93      0.93      2560

